# Version 3

The basic idea here is to extract the CPC first. This is the `core propositional content` or the the central thing that the speaker is talking about -- Could be an action or concept. In either case, it is the thing that the listener is not exepected to have presupposed. E.g., "The purple box is open". There the listener is expected to know about the purple box, but not that it is open. 

Algorithm (Based on the notion of "Quinian Bootstrapping") 
1. Tethering to an LLM's conceptual knowledge
    1. Classify speech act
    2. Extract cpc
    3. Tethering cpc symbol to conceptual commonsense knowledge. Look up typical possible predicate-arguments for cpc, and generate a set of candidate CPC predicate definitions 
        - name(arg_type1, arg_type2)
        - If multiple, then keep them all. 
    4. Extract SPCs
        - what are the central referents likely associated with the identified CPCs. Which CPCs are likely going to work 
        - supporting referents
        - properties of each
        - relations between each 
    5. What we get is a candidate set of CPC predicate definitions and SPC definitions. Specifically, we get a set of `placeholder symbols` that we must next relate to known symbols.  

2. Tethering to Grounded Capabilities --> through constraint sat.
    1. I